# Create Environment

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import emd

In [ ]:
# run tests
emd.support.run_tests()

# 1: Demixing Nonstationary Coupled Oscillators

## EMD
- Decompose signal into IMFs
- Compute the instantaneous phase, frequency, and amplitude of IMFs
- plot instantaneous frequency and amplitude of fast oscillation against phase of slow oscillation

In [ ]:
%matplotlib inline

# a) generate oscillator with two frequencies
T = 10 # duration in seconds
fs = 1000 # sampling frequency in Hz
dt = 1/fs
t = np.arange(0, T, dt) # time vector
if_slow = 5 + 2*np.sin(2*np.pi*0.5*t) # frequency 1 that oscillates with 0.5 Hz around 5 Hz with amplitude 2 Hz
ip_slow = 2 * np.pi * np.cumsum(if_slow) * dt # ip_slow = 2*pi int(if_slow dt)
amp_slow = 5
ip_slow = np.mod(ip_slow, 2*np.pi) # wrap phase [0, 2*pi]
slow_sig = amp_slow * np.cos(ip_slow)

# amp_fast and if_fast are modulate by ip_slow, at peaks of ip_slow
amp_fast = 2 + 2*np.sin(ip_slow + np.pi/2)
if_fast = 25 + 0.5*np.sin(ip_slow)
ip_fast = 2 * np.pi * np.cumsum(if_fast) * dt
fast_sig = amp_fast * np.cos(ip_fast)

# combine slow and fast oscillations
sig = slow_sig + fast_sig
imfs = np.array([fast_sig, slow_sig]).T

fig, ax = plt.subplots(figsize=(15, 10))
emd.plotting.plot_imfs(imfs, t, fig=fig, ax=ax)
ax.set_title('Ground Truth IMFs')
plt.show()

# b) EMD
# get IMFs
sig_imfs = emd.sift.sift(sig)

# plot imfs
fig, ax = plt.subplots(figsize=(15, 10))
emd.plotting.plot_imfs(sig_imfs, t, fig=fig, ax=ax)
ax.set_title('IMFs from EMD')
plt.show()

# c) Hilbert Transform
# get instantaneous phase, frequency, and amplitude via hilbert transform
IP, IF, IA = emd.sift.frequency_transform(sig_imfs, fs, 'hilbert')

# get hilert-huang transform
f, hht = emd.spectra.hilberthuang(IF, IA, edges=np.arange(1, 30, 0.5), sample_rate=fs, sum_time=False)

# plot hht
fig, ax = plt.subplots()
emd.plotting.plot_hilberthuang(hht, t, f, ax=ax, cmap='viridis')
plt.show()

# d) Modulation By Slow Phase
# plot instantaneous frequency and amplitude of fast oscillation as a function of the phase of the slow oscillation
fig = plt.figure()
plt.subplot(2,1,1)
plt.scatter(ip_slow, if_fast)
plt.ylabel('Instantaneous frequency (Hz)')
plt.subplot(2,1,2)
plt.scatter(ip_slow, amp_fast)
plt.ylabel('Amplitude')
plt.xlabel('Phase of slow oscillation')
fig.suptitle('Modulation of fast oscillation by slow oscillation')
plt.show()